In [9]:
import torch
import dgl
import dgl.function as fn
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph

In [10]:
class GCNLayer(nn.Module):
    def __init__(self, layer_input, layer_output):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(layer_input, layer_output)

    def forward(self, g, feature):
        # Creating a local scope so that all the stored ndata and edata
        # (such as the `'h'` ndata below) are automatically popped out
        # when the scope exits.
        # g.ndata: nodes
        # g.edata: edges
        with g.local_scope():
            g.ndata['h'] = feature
            g.update_all(fn.copy_src(src='h', out='m'), fn.sum(msg='m', out='h'))
            h = g.ndata['h']
            return self.linear(h)

In [12]:
class GCN(nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        # Initialize the layers
        # Input feature size: 1433
        # Output: 7 labels (classes)
        self.layer1 = GCNLayer(1433,16)
        self.layer2 = GCNLayer(16, 7)

    def forward(self, g, features):
        # ReLU as activation function
        x = F.relu(self.layer1(g, features))
        x = self.layer2(g, x)
        return x


net = GCN()
print(net)

GCN(
  (layer1): GCNLayer(
    (linear): Linear(in_features=1433, out_features=16, bias=True)
  )
  (layer2): GCNLayer(
    (linear): Linear(in_features=16, out_features=7, bias=True)
  )
)


In [50]:
from dgl.data import citation_graph as citegrh
import networkx as nx

def load_cora_data():
    data = citegrh.load_cora()
    features = torch.Tensor(data.features)
    labels = torch.LongTensor(data.labels)
    train_mask = torch.BoolTensor(data.train_mask)
    test_mask = torch.BoolTensor(data.test_mask)
    g = DGLGraph(data.graph)
    return g, features, labels, train_mask, test_mask

In [51]:
def evaluate(model, g, features, labels, test_mask):
    # Enter eval mode instead of training mode
    model.eval()

    # Deactivate autograd engine, reduce memory and speed up computations
    # Generally always recommended for validation
    with torch.no_grad():
        logits = model(g, features)
        logits = logits[test_mask]
        labels = labels[test_mask]
        # Get index of the highest value (highest probability of prediction)
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() / len(labels)

In [52]:
import time
import numpy as np

# Load the data
g, features, labels, train_mask, test_mask = load_cora_data()

# Initialize the optimizer
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)

for epoch in range(50):

    # Enter training mode
    net.train()

    # Vector of raw non-normalized predictions
    logits = net(g, features)

    normalized = F.log_softmax(logits, 1)
    loss = F.nll_loss(normalized[train_mask], labels[train_mask])

    # Default action: sum the loss.backward() which is only useful in cases like RNNs
    # Call zero_grad() to reset the gradients to 0 on every loop
    optimizer.zero_grad()

    # Computes gradient of loss w.r.t all the parameters in the loss
    loss.backward()

    # Updates all the parameters based on the newly computed gradient
    optimizer.step()

    acc = evaluate(net, g, features, labels, test_mask)
    print("Epoch {:05d} | Loss {:.4f} | Test Acc {:.4f}".format(epoch, loss.item(), acc))





Epoch 00000 | Loss 0.0112 | Test Acc 0.7730
Epoch 00001 | Loss 0.0428 | Test Acc 0.7710
Epoch 00002 | Loss 0.0161 | Test Acc 0.7580
Epoch 00003 | Loss 0.0254 | Test Acc 0.7520
Epoch 00004 | Loss 0.0261 | Test Acc 0.7580
Epoch 00005 | Loss 0.0133 | Test Acc 0.7630
Epoch 00006 | Loss 0.0129 | Test Acc 0.7630
Epoch 00007 | Loss 0.0194 | Test Acc 0.7610
Epoch 00008 | Loss 0.0201 | Test Acc 0.7640
Epoch 00009 | Loss 0.0153 | Test Acc 0.7580
Epoch 00010 | Loss 0.0106 | Test Acc 0.7500
Epoch 00011 | Loss 0.0128 | Test Acc 0.7440
Epoch 00012 | Loss 0.0163 | Test Acc 0.7440
Epoch 00013 | Loss 0.0153 | Test Acc 0.7460
Epoch 00014 | Loss 0.0116 | Test Acc 0.7560
Epoch 00015 | Loss 0.0106 | Test Acc 0.7570
Epoch 00016 | Loss 0.0128 | Test Acc 0.7560
Epoch 00017 | Loss 0.0140 | Test Acc 0.7540
Epoch 00018 | Loss 0.0127 | Test Acc 0.7540
Epoch 00019 | Loss 0.0106 | Test Acc 0.7430
Epoch 00020 | Loss 0.0107 | Test Acc 0.7420
Epoch 00021 | Loss 0.0122 | Test Acc 0.7420
Epoch 00022 | Loss 0.0124 | Test

In [92]:
g, features, labels, train_mask, test_mask = load_cora_data()
print(train_mask[0])

tensor(True)


In [91]:
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)

net.train()

logits = net(g, features)

normalized = F.log_softmax(logits, 1)
loss = F.nll_loss(normalized[train_mask], labels[train_mask])

optimizer.zero_grad()

loss.backward()

optimizer.step()

net.eval()


IndexError: The shape of the mask [2708] at index 0 does not match the shape of the indexed tensor [1000] at index 0

In [90]:
with torch.no_grad():
    logits = net(g, features)
  #  logits = logits[test_mask]
  #  labels = labels[test_mask]
   # _, indices = torch.max(logits, dim=1)
   # correct = torch.sum(indices == labels)
   # print(correct)

IndexError: The shape of the mask [2708] at index 0 does not match the shape of the indexed tensor [1000] at index 0

In [74]:
index = 0
for i in test_mask:
    print(i)
    print(index)
    index = index+1
    if index == 700:
        break

tensor(False)
0
tensor(False)
1
tensor(False)
2
tensor(False)
3
tensor(False)
4
tensor(False)
5
tensor(False)
6
tensor(False)
7
tensor(False)
8
tensor(False)
9
tensor(False)
10
tensor(False)
11
tensor(False)
12
tensor(False)
13
tensor(False)
14
tensor(False)
15
tensor(False)
16
tensor(False)
17
tensor(False)
18
tensor(False)
19
tensor(False)
20
tensor(False)
21
tensor(False)
22
tensor(False)
23
tensor(False)
24
tensor(False)
25
tensor(False)
26
tensor(False)
27
tensor(False)
28
tensor(False)
29
tensor(False)
30
tensor(False)
31
tensor(False)
32
tensor(False)
33
tensor(False)
34
tensor(False)
35
tensor(False)
36
tensor(False)
37
tensor(False)
38
tensor(False)
39
tensor(False)
40
tensor(False)
41
tensor(False)
42
tensor(False)
43
tensor(False)
44
tensor(False)
45
tensor(False)
46
tensor(False)
47
tensor(False)
48
tensor(False)
49
tensor(False)
50
tensor(False)
51
tensor(False)
52
tensor(False)
53
tensor(False)
54
tensor(False)
55
tensor(False)
56
tensor(False)
57
tensor(False)
58
tensor(

In [64]:
print(test_mask)

tensor([False, False, False,  ..., False, False, False])


In [55]:
print(logits)

tensor([[-34.2109, -45.7060,  31.5092,  ..., -14.0037, -43.8658, -41.0090],
        [  3.5579, -12.0472,  -4.9168,  ...,  -0.4658,  24.3183,  -0.7658],
        [-43.2300, -67.4477, -26.2306,  ...,  39.6008, -43.9652, -27.7014],
        ...,
        [ -8.7316,   2.7198,  -3.9358,  ...,  -2.7140, -11.8802,  -8.9091],
        [ 15.8358, -18.4486,  -4.7200,  ...,  -4.0193, -11.3843, -11.6701],
        [ -6.0200,  -8.7746,  -0.5245,  ...,  -2.7529,  -5.3617,  -7.4319]])


In [75]:
print(logits[500])

tensor([-13.4219,  -9.1178, -11.0920,  -9.3397,   9.2593, -20.2658, -16.4854])


In [60]:
print(logits[test_mask])


tensor([[-13.4219,  -9.1178, -11.0920,  ...,   9.2593, -20.2658, -16.4854],
        [ -8.7528, -12.3523,  -0.9439,  ...,  -1.8563, -15.2689, -11.4883],
        [-58.2328, -70.2535,  -4.2399,  ...,  33.0059, -38.5081, -42.9501],
        ...,
        [ 26.7022, -20.2669,  -6.5791,  ...,  -4.7432, -12.3335, -13.2885],
        [  9.9518,  39.0246, -53.8996,  ..., -13.7839, -56.0227, -48.8414],
        [-14.9965, -40.3553,  18.9230,  ...,  -6.4413, -30.7893, -22.3310]])


In [49]:
_, indices = torch.max(logits[test_mask], dim=1)
print(indices)

tensor([4, 2, 4, 2, 3, 2, 6, 6, 6, 5, 2, 2, 2, 3, 6, 3, 2, 6, 3, 2, 6, 5, 3, 4,
        6, 3, 2, 2, 3, 3, 1, 3, 3, 3, 0, 6, 2, 1, 1, 0, 0, 0, 2, 0, 2, 4, 1, 2,
        4, 6, 6, 0, 4, 2, 4, 6, 6, 3, 3, 4, 3, 2, 4, 1, 4, 4, 4, 4, 0, 6, 2, 2,
        2, 3, 6, 4, 2, 3, 0, 2, 0, 0, 0, 0, 2, 5, 2, 2, 4, 2, 3, 6, 2, 0, 0, 2,
        2, 5, 5, 4, 2, 4, 1, 0, 2, 2, 1, 6, 2, 4, 2, 2, 0, 2, 6, 6, 2, 2, 6, 3,
        4, 4, 3, 3, 1, 1, 2, 2, 4, 0, 3, 5, 6, 4, 5, 6, 4, 3, 5, 3, 6, 3, 3, 0,
        4, 3, 3, 3, 6, 0, 4, 6, 5, 4, 5, 2, 4, 4, 1, 1, 6, 3, 3, 2, 2, 2, 2, 2,
        3, 3, 0, 4, 6, 3, 2, 5, 2, 3, 3, 0, 2, 1, 0, 5, 3, 2, 6, 2, 4, 1, 0, 4,
        6, 2, 3, 4, 2, 3, 2, 4, 2, 5, 3, 4, 3, 2, 6, 5, 2, 3, 6, 3, 1, 3, 1, 3,
        2, 1, 2, 2, 6, 1, 6, 4, 2, 3, 2, 1, 1, 3, 2, 2, 6, 4, 3, 5, 2, 2, 4, 2,
        6, 2, 3, 1, 1, 2, 2, 2, 3, 4, 4, 2, 0, 4, 0, 1, 3, 1, 1, 3, 0, 6, 0, 2,
        4, 1, 6, 6, 3, 1, 2, 2, 3, 4, 2, 4, 3, 3, 6, 2, 1, 1, 3, 0, 3, 0, 6, 3,
        3, 1, 0, 6, 6, 1, 6, 3, 3, 2, 6,

In [66]:
for i in labels:
    print(i)

tensor(2)
tensor(2)
tensor(6)
tensor(3)
tensor(1)
tensor(3)
tensor(1)
tensor(3)
tensor(2)
tensor(1)
tensor(2)
tensor(2)
tensor(2)
tensor(1)
tensor(6)
tensor(3)
tensor(2)
tensor(3)
tensor(2)
tensor(1)
tensor(1)
tensor(3)
tensor(2)
tensor(2)
tensor(6)
tensor(4)
tensor(4)
tensor(5)
tensor(2)
tensor(2)
tensor(4)
tensor(4)
tensor(6)
tensor(2)
tensor(3)
tensor(1)
tensor(1)
tensor(2)
tensor(2)
tensor(2)
tensor(3)
tensor(4)
tensor(4)
tensor(2)
tensor(0)
tensor(2)
tensor(2)
tensor(1)
tensor(5)
tensor(1)
tensor(1)
tensor(3)
tensor(0)
tensor(6)
tensor(6)
tensor(2)
tensor(1)
tensor(1)
tensor(2)
tensor(6)
tensor(3)
tensor(1)
tensor(2)
tensor(2)
tensor(3)
tensor(4)
tensor(2)
tensor(4)
tensor(3)
tensor(3)
tensor(6)
tensor(2)
tensor(1)
tensor(1)
tensor(2)
tensor(0)
tensor(2)
tensor(0)
tensor(6)
tensor(3)
tensor(3)
tensor(1)
tensor(6)
tensor(6)
tensor(6)
tensor(1)
tensor(6)
tensor(6)
tensor(2)
tensor(2)
tensor(6)
tensor(0)
tensor(1)
tensor(0)
tensor(3)
tensor(1)
tensor(4)
tensor(2)
tensor(0)
tensor(4)


In [76]:
print(labels[500])

tensor(4)


In [47]:
print(labels[test_mask])

tensor([4, 2, 4, 2, 2, 2, 4, 6, 6, 5, 3, 3, 2, 3, 6, 3, 2, 6, 6, 2, 2, 5, 6, 1,
        2, 6, 2, 2, 2, 2, 1, 2, 3, 3, 0, 6, 2, 1, 1, 0, 0, 0, 2, 0, 2, 2, 1, 2,
        4, 6, 6, 4, 4, 2, 4, 6, 6, 3, 3, 4, 6, 2, 4, 4, 4, 4, 4, 4, 4, 6, 2, 6,
        2, 3, 6, 4, 2, 3, 0, 2, 0, 0, 0, 0, 3, 5, 2, 2, 4, 2, 2, 6, 2, 0, 0, 2,
        6, 5, 5, 2, 2, 4, 1, 0, 2, 2, 1, 2, 2, 2, 2, 2, 0, 2, 6, 6, 2, 2, 6, 3,
        2, 4, 2, 3, 6, 2, 2, 2, 2, 0, 2, 2, 3, 4, 0, 6, 4, 3, 6, 3, 6, 3, 3, 0,
        4, 3, 3, 3, 2, 0, 4, 2, 5, 4, 5, 2, 4, 4, 1, 2, 6, 3, 2, 2, 2, 2, 2, 2,
        2, 5, 0, 4, 6, 2, 2, 5, 2, 2, 2, 0, 2, 1, 4, 0, 3, 2, 0, 2, 4, 1, 0, 4,
        6, 2, 2, 6, 2, 3, 2, 4, 6, 2, 2, 2, 3, 2, 6, 5, 2, 2, 6, 3, 1, 3, 1, 3,
        2, 1, 2, 2, 2, 1, 6, 3, 2, 3, 2, 1, 1, 3, 2, 2, 6, 4, 4, 5, 2, 2, 4, 4,
        6, 2, 3, 1, 1, 2, 2, 2, 3, 4, 4, 2, 0, 2, 2, 1, 5, 1, 1, 3, 0, 6, 6, 2,
        1, 1, 2, 6, 3, 1, 2, 2, 3, 4, 2, 4, 3, 3, 6, 2, 1, 1, 2, 0, 2, 0, 6, 3,
        3, 1, 6, 6, 6, 1, 6, 6, 2, 2, 6,